# Libraries

In [1]:
import auth.gmail_auth
import utils.html_module 
import scraping.scrap_job_blocks
import scraping.scrap_job_elements
import processing.latest_emails
import scraping.overall_scrap
import pandas as pd

In [2]:
import logging

logging.basicConfig(level=logging.INFO)

In [3]:
import os
import pickle
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from bs4 import BeautifulSoup
import requests
import webbrowser
from tempfile import NamedTemporaryFile
import re

# Google Authentication

In [4]:
try:
    service = auth.gmail_auth.authenticate_gmail()
    print("Gmail API authenticated successfully!")
except Exception as e:
    print("Error during authentication:", e)

INFO:googleapiclient.discovery_cache:file_cache is only supported with oauth2client<4.0.0


Gmail API authenticated successfully!


# Sample email saved as html webpage 
So we can judge the struture and write the logic

In [5]:
# Fetch the email and save it as an HTML file
utils.html_module.fetch_email_and_save_as_html(service)


Email saved as src/usecase_most_recent/indeed_alert.html. Open this file in a browser to examine it.


# Parsing Local and online emails

####  3 use cases files
v1 offline files 
- intial email - offline, 
- subsiquest email - offiline
  
Most recent
- current most recent - Fetch at the moment then made offline

In [6]:
initial_email_local = utils.html_module.parse_local_html("src/usecases_v1_offline/indeed_alert_initial.html")
initial_email_subsiquest = utils.html_module.parse_local_html("src/usecases_v1_offline/indeed_alert_subsiquest.html")
online_email = utils.html_module.fetch_email_html_content_most_recent_only(service)

Local HTML file parsed successfully.
Local HTML file parsed successfully.
Email HTML content fetched and parsed successfully.


In [7]:
# online_email

In [8]:
no_job_edge_case = utils.html_module.parse_local_html("src/usecases_v1_offline/indeed_alert_edge_case_no_job_in_alert.html")


Local HTML file parsed successfully.


In [9]:
# All types are sounp object 
type(initial_email_local)
type(initial_email_subsiquest)
type(online_email)

bs4.BeautifulSoup

In [10]:
# Look at the HTML of each from here 
# If Required:
    # copy to https://html.onlineviewer.net/ to understand more
    # Copy the content to file renamd it .html and see 

## Uncoment below so check
# initial_email_local # Fine
# initial_email_subsiquest # Fine 
# online_email # fine

In [11]:
# Find all specific type of the tag 
initial_email_subsiquest.find_all('table') # change the object to explore other use cases 

# put those type of tags in list 
list_of_tags = list(initial_email_subsiquest)  # Explicitly converting to a list
# tbodies_list # for all objects
# list_of_tags

# For a single tag 
# list_of_tags[1] # change index to explore

# Scrapping mindset
Extract job postings by leveraging relative positions and contextual cues from the provided static HTML files. just the dynamic words based on static words.

#### Key Point:
Rely on positional relationships and contextual keywords, not static tags or class names.


# 1. List of Job Blocks Fetch 

Identify elements containing text like "X days ago" or "Just posted" as anchors. 



In [12]:
# This extracts the individual job posts blocks

job_blocks_initial_email_local = scraping.scrap_job_blocks.extract_individual_job_blocks(initial_email_local)
job_blocks_subsiquest_email_local =  scraping.scrap_job_blocks.extract_individual_job_blocks(initial_email_subsiquest)
job_blocks_online_email = scraping.scrap_job_blocks.extract_individual_job_blocks(online_email)

type(job_blocks_initial_email_local) # list

list

#### Opening in browser to see how do they look

try looking for mupliple indexes

In [13]:
# utils.html_module.open_html_in_browser(utils.html_module.render_html(job_blocks_initial_email_local[2]))
# utils.html_module.open_html_in_browser(utils.html_module.render_html(job_blocks_subsiquest_email_local[3]))
# utils.html_module.open_html_in_browser(utils.html_module.render_html(job_blocks_online_email[1]))

# 2. Scrapping individual Job Post data

Fetch
- Title
- Company
- Posted Days 


In [14]:
type(job_blocks_online_email[2])

bs4.element.Tag

In [15]:
soup = job_blocks_online_email[1]

In [16]:
rows = soup.find_all('tr')

# Prepare data for the DataFrame
# Prepare data for the DataFrame with an additional "Link" column
data = []
for idx, row in enumerate(rows, start=1):  # Start numbering at 1
    columns = [td.text.strip() for td in row.find_all('td')]
    link = row.find('a', href=True)  # Find the first <a> tag with href attribute
    row_data = {
        "Number": idx,
        "TR HTML": str(row),
        "Link": link['href'] if link else None,  # Add the link if present
    }
    # Dynamically add data columns
    for i, col in enumerate(columns, start=1):
        row_data[f"Data {i}"] = col
    data.append(row_data)

# Create the DataFrame
df = pd.DataFrame(data)

# Display the DataFrame

df

,Number,TR HTML,Link,Data 1,Data 2,Data 3,Data 4
0,1,"<tr><td align=""left"" valign=""top""><h2 style=""c...",https://ca.indeed.com/rc/clk/dl?jk=933e47c1319...,"Consultant, Risk Analytics (10 M Secondment)",NaN,NaN,NaN
1,2,"<tr><td align=""left"" valign=""top""><table borde...",None,CIBC3.7,CIBC,3.7,
2,3,"<tr><td style=""padding:0 12px 0 0;color:#2d2d2...",None,CIBC,3.7,,NaN
3,4,"<tr><td align=""left"" style=""color:#2d2d2d;font...",None,"Toronto, ON",NaN,NaN,NaN
4,5,"<tr><td align=""left"" style=""padding:0;color:#7...",None,Use data-driven processes to effectively manag...,NaN,NaN,NaN
5,6,"<tr><td align=""left"" style=""padding:8px 0 0;co...",None,2 days ago,NaN,NaN,NaN


In [17]:
scraping.scrap_job_elements.get_individual_job(job_blocks_online_email[1])

,title,link,company,rating,location,type,description,days_posted,days,posting_date,fetched_date
0,"Consultant, Risk Analytics (10 M Secondment)",https://ca.indeed.com/rc/clk/dl?jk=933e47c1319...,CIBC,3.7,"Toronto, ON",None,Use data-driven processes to effectively manag...,2 days ago,2,2024-12-21,2024-12-23


# Scrapping all jobs at once, as dataframe from single file

In [18]:
all_jobs_from_a_email = scraping.overall_scrap.scrap_all_individual_jobs(job_blocks_online_email)
all_jobs_from_a_email

,title,link,company,rating,location,type,description,days_posted,days,posting_date,fetched_date
0,"Senior Risk Analyst, Regulatory Reporting & An...",https://ca.indeed.com/rc/clk/dl?jk=fc622f9c1db...,CIBC,3.7,"Toronto, ON",Hybrid remote,You also have some experience in the areas of ...,1 day ago,1,2024-12-22,2024-12-23
1,"Consultant, Risk Analytics (10 M Secondment)",https://ca.indeed.com/rc/clk/dl?jk=933e47c1319...,CIBC,3.7,"Toronto, ON",None,Use data-driven processes to effectively manag...,2 days ago,2,2024-12-21,2024-12-23
2,"Senior Analyst, Business Controls",https://ca.indeed.com/rc/clk/dl?jk=853a47848de...,CIBC,3.7,"Toronto, ON",Hybrid remote,"Reporting to the Senior Manager, Central Regul...",4 days ago,4,2024-12-19,2024-12-23
3,Senior AI Data Scientist,https://ca.indeed.com/rc/clk/dl?jk=67b7df3d1b0...,CIBC,3.7,"Toronto, ON",Hybrid remote,The Decision Science team is a central hub of ...,1 day ago,1,2024-12-22,2024-12-23
4,"Analyst, Global Markets",https://ca.indeed.com/rc/clk/dl?jk=0be29db6266...,CIBC,3.7,"Toronto, ON",None,You give meaning to data. You will support the...,2 days ago,2,2024-12-21,2024-12-23
5,"Sr. Consultant, Data Security Operations",https://ca.indeed.com/rc/clk/dl?jk=2ccb9d02956...,CIBC,3.7,"Toronto, ON",Hybrid remote,You have been a key contributor or have led da...,5 days ago,5,2024-12-18,2024-12-23
6,"Analyst, Global Mining & Forestry, Corporate B...",https://ca.indeed.com/rc/clk/dl?jk=dea2a483946...,CIBC,3.7,"Toronto, ON",None,As an Analyst on the Global Mining & Forestry ...,5 days ago,5,2024-12-18,2024-12-23
7,"Sr. Financial Analyst, Treasury Product Control",https://ca.indeed.com/rc/clk/dl?jk=96a60b177c8...,CIBC,3.7,"Toronto, ON",Hybrid remote,Stay current on tools and information availabl...,3 days ago,3,2024-12-20,2024-12-23


# Create and updating monthly csv raw processed file for conolidated raw monthly data 

In [19]:
scraping.overall_scrap.results_create_or_append_to_csv(all_jobs_from_a_email, reset_file = False)

Data appended to ./data/raw_processed\2024_12.csv.


So Basically
- frist get => scraping.scrap_job_blocks.extract_individual_job_blocks(soup)
- 2nd get =>  scraping.scrap_job_elements.scrap_all_jobs(list of soups) => data frame of all jobs as dataframe
- 3rd perfrom consolidation => scraping.scrap_job_elements.create_or_append_to_csv(all_jobs_from_a_email, reset_file = False)
  which is basiclly a runnig raw file for a month 


In [20]:
# Functiion for all steps togther
scraping.overall_scrap.scrap_process_email_content_to_csv(initial_email_local)

INFO:root:Successfully processed email content for job data.


Data appended to ./data/raw_processed\2024_12.csv.


# Next, encoprate this in the live email fetch 

# --------------------------------------------------------------

# On email receive trigger the parser - Active Listner
This listens to all the new eamil from a spcified sender

In [21]:
# import time
# from googleapiclient.discovery import build
# from googleapiclient.errors import HttpError

# # Define the sender and query for filtering
# SEARCH_SENDER = "malikhq27@gmail.com"

# def list_new_emails(service):
#     """List new emails from a specific sender and print their titles."""
#     try:
#         # Search for emails matching the query (e.g., from:alert@indeed.com)
#         query = f"from:{SEARCH_SENDER} is:unread"
#         results = service.users().messages().list(userId='me', q=query).execute()
#         messages = results.get('messages', [])

#         if not messages:
#             print("No new emails found.")
#             return

#         print(f"Found {len(messages)} new email(s) from {SEARCH_SENDER}.")
#         for message in messages:
#             # Fetch email details
#             msg = service.users().messages().get(userId='me', id=message['id']).execute()
#             headers = msg['payload']['headers']

#             # Extract subject from email headers
#             subject = next((header['value'] for header in headers if header['name'] == 'Subject'), "No Subject")
#             print(f"New Email Title: {subject}")

#             # Mark the email as read (optional)
#             service.users().messages().modify(
#                 userId='me',
#                 id=message['id'],
#                 body={"removeLabelIds": ["UNREAD"]}
#             ).execute()

#     except HttpError as error:
#         print(f"An error occurred: {error}")


# if __name__ == '__main__':
#     try:
#         # Authenticate using Gmail API
#         service = authenticate_gmail()
#         print("Gmail API authenticated successfully!")

#         # Continuously check for new emails
#         while True:
#             list_new_emails(service)
#             print("Waiting for new emails...")
#             time.sleep(10)  # Poll every 10 seconds

#     except Exception as e:
#         print("Error:", e)
